![](../resources/20251020_Spaceborne_EO_tech_school.png)

### Content

- [Introduction](###-Introduction)
- [How to run this notebook?](###-How-to-run-this-notebook?)
- [Before you start](###-Before-you-start)
- [1. Gather and prepare your training data](###-1.-Gather-and-prepare-your-training-data)
- [2. Train custom classification model](###-2.-Train-custom-classification-model)
- [3. Deploy your custom model](###-3.-Deploy-your-custom-model)
- [4. Testing your model locally](###-4.-Testing-your-model-locally)
- [5. BONUS: Apply your model anywhere!](###-5.-BONUS:-Apply-your-model-anywhere!)

### Introduction

This notebook guides you through the process of training a custom crop type classification model using the WorldCereal crop mapping system.<br>

For today's exercise we'll be constructing a model based on reference data from a data dense region (France) and apply this model to an area in Czech Republic to see how tranferable our model is across space.

<div style="background-color:#fff3cd; border-left:5px solid #ffeeba; padding:10px; color:#856404;">
This colored box is used throughout the notebook to highlight questions, tasks, or reflections for you to consider while completing the exercise.<br>
GOOD LUCK!
</div>

### How to run this notebook?

You can use a preconfigured environment on [**Terrascope**](https://terrascope.be/en) to run the workflows in a Jupyter notebook environment.<br>
Just register as a new user on Terrascope or use one of the supported EGI eduGAIN login methods to get started.

Once you have a Terrascope account, you can run this notebook by clicking the button shown below.

<div class="alert alert-block alert-warning">
<b>WARNING:</b> <br>
When you click the button, you will be prompted with "Server Options".<br>
Make sure to select the "Worldcereal" image here. Did you choose "Terrascope" by accident?<br>
Then go to File > Hub Control Panel > Stop my server, and click the link below once again.</div>


<a href="https://notebooks.terrascope.be/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2FWorldCereal%2Fworldcereal-classification&urlpath=lab%2Ftree%2Fworldcereal-classification%2Fnotebooks%2Ftrainings%2F20251020_Spaceborne_EO_tech_agri.ipynb&branch=main"><img src="https://img.shields.io/badge/Run%20exercise%20on-Terrascope-brightgreen" alt="Generate custom crop type map" valign="middle"></a>


<div class="alert alert-block alert-warning">
<b>WARNING:</b> <br>
Every time you click the above link, the latest version of the notebook will be fetched, potentially leading to conflicts with changes you have made yourself.<br>
To avoid such code conflicts, we recommend you to make a copy of the notebook and make changes only in your copied version.
</div>

### Before you start

In order to run WorldCereal crop mapping jobs from this notebook, you need to create an account on the [Copernicus Data Space Ecosystem](https://dataspace.copernicus.eu/).<br>
This is free of charge and will grant you a number of free openEO processing credits to continue this demo.

Execute the next block of code to ensure this notebook has access to all functionalities needed in this exercise.

In [ ]:
# add parent dirctory to sys.path
import sys
sys.path.append('..')

### 1. Gather and prepare your training data

For training a crop type model, you can use a combination of:
- publicly available reference data harmonized by the WorldCereal consortium;
- your own private reference data (not part of today's exercise).

The cell below provides you with a quick overview of the publicly exposed reference datasets for which WorldCereal has already done satellite extractions. Hence these are ready to be plugged into model training. In the next few weeks, this extractions database will receive a major update, adding many more datasets for you to use in model training.


<div class="alert alert-block alert-info">
<b>Note on reference data availability:</b><br>

For a detailed exploration of available reference data for your region of interest, you can:

- use the WorldCereal Reference Data Module user interface, available [here](https://rdm.esa-worldcereal.org/). More explanation can be found [here](https://worldcereal.github.io/worldcereal-documentation/rdm/explore.html#explore-data-through-our-user-interface).
- use our dedicated notebook [worldcereal_RDM_demo.ipynb](https://github.com/WorldCereal/worldcereal-classification/blob/main/notebooks/worldcereal_RDM_demo.ipynb).
</div>



In [ ]:
from notebook_utils.extractions import retrieve_extractions_extent

extents = retrieve_extractions_extent()
print(f"Found {len(extents)} datasets with extractions.")
extents.explore(
            style_kwds={
                "fillOpacity": 0.05,  # Transparency of polygon fill (0 = fully transparent, 1 = opaque)
                "weight": 1,  # Border line width
            },
            highlight=False,
        )

**Step 1: Select your area of interest (AOI)**

Provide a bounding box specifying the region in which you would like to look for available reference data.<br>

When running the code snippet below, an interactive map will be visualized.<br>
Click the Rectangle button on the left hand side of the map to start drawing your region of interest.<br>
The widget will automatically store the coordinates of the last rectangle you drew on the map.<br>

<div style="background-color:#fff3cd; border-left:5px solid #ffeeba; padding:10px; color:#856404;">
<b>Your turn:</b><br>
For the purpose of this exercise, select a bounding box in France, not exceeding 1000 km².<br>
(the larger the area, the longer the request for reference data will take...)
</div>

In [ ]:
from worldcereal.utils.map import ui_map

map = ui_map(area_limit=1000)

**Step 2: Get all available reference data**

You can now query both public and private extractions and retrieve the relevant samples based on your defined area of interest.<br>
By default, a spatial buffer of 250 km is applied to your area of interest to ensure sufficient training data is found.<br>
You can freely expand this search perimeter by changing the value of the `buffer` parameter.

<div class="alert alert-block alert-info">
<b>Note on the use of private data:</b><br>
In case you would like to include your private training data, make sure to first prepare the necessary extractions through our dedicated notebook --> <b><a href=https://github.com/WorldCereal/worldcereal-classification/blob/main/notebooks/worldcereal_private_extractions.ipynb>worldcereal_private_extractions.ipynb.</a></b><br>

Then, specify the private_extractions_path in the cell below, where your private extractions reside!
</div>

<div class="alert alert-block alert-info">
<b>Important consideration on model scope!</b><br>

By default, we filter the reference data explicitly to only retain temporary crops, by setting the `filter_temporary_crops` parameter to `True`.<br>
This effectively means that you will (by default) only be able to train a model distinguishing different types of temporary crops.<br>
In case you would like to expand your scope towards other land cover and/or permanent crops, please set `filter_temporary_crops` to `False`.
</div>

<div style="background-color:#fff3cd; border-left:5px solid #ffeeba; padding:10px; color:#856404;">
For this exercise, we only use public data on temporary crops<br>
The datasets in France contain many crop types. We have limited the query to a fixed number of crops to not make things overly complex.<br>
No need to change anything in the next cell.
</div>

In [ ]:
from pathlib import Path
from notebook_utils.extractions import query_extractions

# Retrieve the polygon you drew on the map
polygon = map.get_polygon_latlon()

# Specify a buffer distance to expand your search perimeter
buffer = 250000  # meters

# Specify the path to the private extractions data; 
# if you followed the private extractions notebook, your extractions path should be the one commented below;
# if you leave this None, only public data will be queried
private_extractions_path = None
# private_extractions_path = Path('./extractions/worldcereal_merged_extractions.parquet')

# Specify whether you are only interested in temporary crops only (True) or all available classes (False)
filter_temporary_crops = True

# For the purpose of this exercise, we limit the number of crops to be considered
crop_types = [1101060000, 1103080080, 1103090060, 1106000031, 1111000010, 1111020010, 1101060001, 1106000020, 1101010011, 1101020001, 1101020002, 1103060000, 1103060040, 1103110040, 1111000000, 1105010010, 1105010020, 1107000010, 1106000030, 1106000010, 1105010040, 1105000030, 1103090040, 1107000031, 1108020010]

# Query our public database of training data
extractions = query_extractions(polygon, buffer, private_parquet_path=private_extractions_path, filter_cropland=filter_temporary_crops, crop_types=crop_types)
extractions.head()

**Step 3: Perform a quick quality check**

In this step, we provide you with some tools to quickly assess the quality of the datasets.

Upon executing this cell, you will be prompted to enter a dataset name (ref_id) for inspection.

Especially the visualization of the time series might help you better define your season of interest later on.

In [ ]:
from notebook_utils.extractions import get_band_statistics, visualize_timeseries

dataset_name = input('Enter the dataset name: ')
subset_data = extractions.loc[extractions['ref_id'] == dataset_name]
# Optionally, filter by crop group:
# subset_data = subset_data[subset_data['sampling_label'] == 'wheat']

# Check band statistics
band_stats = get_band_statistics(subset_data)

# Visualize NDVI timeseries for a few samples
visualize_timeseries(subset_data, band='NDVI', nsamples=3, random_seed=42)

<div style="background-color:#fff3cd; border-left:5px solid #ffeeba; padding:10px; color:#856404;">
Optionally also try to visualize the Sentinel-1 VV band using the visualize_timeseries function.
</div>

**Step 4: Select your season of interest**

Keep in mind that in WorldCereal, we train **season-specific** crop classifiers.<br>
In this step, you are asked to specify your cropping season of interest.<br>
Based on this information, we get rid of irrelevant training data and prepare the classification features in the next step.<br>

To gain a better understanding of crop seasonality in your area of interest, you can consult the WorldCereal crop calendars (by executing the next cell), or check out the [USDA crop calendars](https://ipad.fas.usda.gov/ogamaps/cropcalendar.aspx).

In [ ]:
from notebook_utils.seasons import retrieve_worldcereal_seasons

spatial_extent = map.get_extent()
seasons = retrieve_worldcereal_seasons(spatial_extent)

Now let's also check the distribution of `valid_time` in your reference data.<br>
This attribute indicates the date for which the crop label is actually valid.<br>
This is important to consider when selecting your season of interest: it does not make too much sense to train a classifier for a season in which you have barely any valid reference data to work with!

In [ ]:
from notebook_utils.seasons import valid_time_distribution

valid_time_distribution(extractions)

Now use the slider to select your season of interest.<br>

Note that WorldCereal models are always trained on time series of **12 months**.<br>
However, the models are instructed to pay the most attention to the center of your specified period.<br>
In practice, this means the best strategy is to make sure the center of your season of interest nicely coincides with the `Season center` as indicated below the slider.<br>
In the future it will be possible to more exactly specify your season of interest.

In [ ]:
from notebook_utils.dateslider import season_slider

slider = season_slider()

**Step 5: Compute training features**

Using a pretrained geospatial foundation model (Presto), we derive training features for each sample in the dataframe resulting from your query. Presto was pre-trained on millions of unlabeled samples around the world and finetuned on global labelled land cover and crop type data from the WorldCereal reference database. The resulting 128 *embeddings* (`presto_ft_0` -> `presto_ft_127`) nicely condense the Sentinel-1, Sentinel-2, meteo timeseries and ancillary data for your season of interest into a limited number of meaningful features which we will use for downstream model training.<br>

Reference data for which the `valid_time` signficantly deviates from the center of your season of interest will be discarded automatically.<br>
Users have the option to increase/decrease the value of `valid_time_buffer` (expressed in months) if they want to be more/less strict in aligning the reference data to their season of interest.<br>
This parameter defines the minimum distance between the valid_time of a sample and the edge of the 12 months extent you have selected.<br>
The `valid_time_buffer` is allowed to vary between 0 and 6 months. By default we use a value of 2 months.<br>

Finally, we provide one more option during training feature computation aimed at increasing temporal robustness of the your final crop model.<br>
This is controlled by the `augment` parameter: when set to `True`, it introduces slight temporal jittering of the processing window, making the model more robust against slight variations in seasonality across different years.<br>
By default, this option is set to `False`, but especially when training a model across multiple years in areas with a lot of reference data, enabling this option could be considered.


In [ ]:
from notebook_utils.classifier import compute_training_features

# Retrieve the date range you just selected
season = slider.get_selected_dates()

# Prepare the training features for the selected season
training_df = compute_training_features(extractions, season, valid_time_buffer=2, augment=False)
training_df.head()

**Step 5: Select your crops of interest**

The following widget will display all available land cover classes and crop types in your training dataframe.

Tick the checkbox for each crop type you wish to explicitly include in your model.<br>
In case you wish to group multiple crops together, just tick the parent node in the hierarchy.

Non-selected crops will be merged together in an `other` class.

After selecting all your crop types of interest, hit the "Apply" button.

<div class="alert alert-block alert-info">
<b>Minimum number of samples:</b><br>
In order to train a model, we recommend a minimum of 30-50 samples to be available for each unique crop type.<br>
</div>


<div style="background-color:#fff3cd; border-left:5px solid #ffeeba; padding:10px; color:#856404;">
The area we'll be applying our model to has some soybeans, wheat and maize, so make sure you at least select these crops!<br>

Choosing target crops is a key exercise when developing a crop mapping algorithm and often requires trial & error to find out whether specific combinations work well or not. There's no golden standard here. Start from a limited set of crops, keeping in mind the user needs and try to get to a working compromise. If a little further in this exercise your model performs badly, consider updating the chosen crop types and train the model again, until you're happy with the result.
</div>


In [ ]:
from notebook_utils.croptypepicker import CropTypePicker

croptypepicker = CropTypePicker(sample_df=training_df, expand=True)

In the next cell, we apply your selection to your training dataframe.<br>
The new dataframe will contain a `downstream_class` attribute, denoting the final label that will be used during model training.<br>

Let's first check which classes ended up in the "other" class:

In [ ]:
from notebook_utils.croptypepicker import apply_croptypepicker_to_df
from worldcereal.utils.legend import translate_ewoc_codes

training_df = apply_croptypepicker_to_df(training_df, croptypepicker)
other_count = training_df.loc[training_df['downstream_class'] == 'other']['ewoc_code'].value_counts()
other_labels = translate_ewoc_codes(other_count.index.tolist())
other_count.to_frame().merge(other_labels, left_index=True, right_index=True)

Based on this list, you might consider dropping some classes.<br>
This can be done by providing the "ewoc_codes" in the following cell.

If the "Other" class becomes too powerful, i.e. containing a wide range of spectrally very diverse crops, it will deteriorate performance of the other classes by dragging a lot of predictive power into this class. In this case, it is better to either remove original classes so they won't influence the model, or add some more dedicated classes to the nomenclature.

In [ ]:
# drop classes
# to_drop = [1111000000, 1111000010, 1111020010, 1101020001]
to_drop = []
if len(to_drop) > 0:
    training_df = training_df.loc[~training_df['ewoc_code'].isin(to_drop)]
training_df['downstream_class'].value_counts()

Finally, you could opt to combine some classes using the code snippet below as an example:

In [ ]:
# combine_classes = {
#     'cereals': ['winter_barley', 'oats', 'millet', 'winter_rye', 'wheat']}
combine_classes = {}
for new_class, old_classes in combine_classes.items():
    training_df.loc[training_df['downstream_class'].isin(old_classes), 'downstream_class'] = new_class

# Report on the contents of the data
training_df['downstream_class'].value_counts()

**Step 6: Save your final training dataframe for future reference**

Upon executing the next cell, you will be prompted to provide a unique name for your dataframe.

In [ ]:
from pathlib import Path
from notebook_utils.classifier import get_input

df_name = get_input("Name dataframe")

training_dir = Path('./training_data')
training_dir.mkdir(exist_ok=True)

outfile = training_dir / f'{df_name}.csv'

if outfile.exists():
    raise ValueError(f"File {outfile} already exists. Please delete it or choose a different name.")

training_df.to_csv(outfile)

print(f"Dataframe saved to {outfile}")

### 2. Train custom classification model

We train a catboost model for the selected crop types.<br> 

By default, we apply **class balancing** to ensure minority classes are not discarded. However, depending on the class distribution this may lead to undesired results. There is no golden rule here. If your main goal is to make sure the most dominant classes in your training data are very precisely identified in your map, you can opt to NOT apply class balancing by setting: `balance_classes=False`. 

Before training, the available training data has been automatically split into a calibration and validation part. The validation report and confusion matrix already provide you with a first idea on your model's performance.<br>
For visualizing the confusion matrix, you have several options through the `show_confusion_matrix` parameter:
- `absolute`: print absolute sample counts in the confusion matrix
- `relative`: plots the normalized confusion matrix
- `none`: do not visualize a confusion matrix

In [ ]:
from notebook_utils.classifier import train_classifier

custom_model, report, confusion_matrix = train_classifier(
    training_df, balance_classes=True, show_confusion_matrix='absolute',
)
print(report)

### 3. Deploy your custom model

Once trained, your model is uploaded to a dedicated S3 bucket on CDSE, so it can be accessed by OpenEO for generating maps (see next step).<br>
Your model is protected using your CDSE credentials and will not be accessible by anyone else.

Upon executing the next cell, you will be prompted to provide a clear and short name for your custom model.

<div class="alert alert-block alert-warning">
<b>Long term storage of your model:</b> <br>
Note that your model is only kept in cloud storage for a limited amount of time. <br>

Make sure to download your model (using the link provided) if you wish to store it for a longer period of time!<br>

In case you would like to use your model for generating maps at a later point in time, you will need to host your model in a publicly available repository, e.g. Google Drive, and replace `model_url` with this public link during model inference.
</div>

<div class="alert alert-block alert-info">
<b>CDSE authentication:</b><br>
After first login, your CDSE credentials will be stored on your machine avoiding the need for repeating authentication in the future. If you would want to switch CDSE account, execute the following lines of code:<br>
<br>
from notebook_utils.openeo import clear_openeo_token_cache<br>
clear_openeo_token_cache()
</div>

In [ ]:
from worldcereal.utils.upload import deploy_model
from openeo_gfmap.backend import cdse_connection
from notebook_utils.classifier import get_input

modelname = get_input("model")
model_url = deploy_model(cdse_connection(), custom_model, pattern=modelname)
print(f"Your model can be downloaded from: {model_url}")

### 4. Testing your model locally

Using your custom model, we now generate a map for our area of interest in Czech Republic.

The cell below references a preprocessed input file that will be downloaded locally the first time you run the workflow. This file contains all necessary features for a small region in Czech Republic and enables you to test your trained model without requiring cloud processing or repeated data downloads. By working with this local dataset, you can quickly validate your model’s performance and iterate on your workflow before scaling up to larger areas or deploying in the cloud.

<div class="alert alert-block alert-info">
<b>Want to create your own patches of local data?</b><br>
Check out <a href="https://github.com/WorldCereal/worldcereal-classification/blob/main/notebooks/worldcereal_preprocessed_inputs.ipynb" target="_blank">this notebook</a> to learn more! 
</div>

<div style="background-color:#fff3cd; border-left:5px solid #ffeeba; padding:10px; color:#856404;">
Investigate the contents of the xarray.Dataset, which contains the preprocessed inputs the model will use to classify cropland and crop types.
</div>


In [ ]:
from pathlib import Path
import xarray as xr
from pyproj import CRS

local_file_path = Path("./local_inference/preprocessed_inputs.nc")
local_file_path.parent.mkdir(exist_ok=True)
if not local_file_path.exists():
    print(f"Downloading demo preprocessed inputs to {local_file_path}...")
    remote_url = "https://artifactory.vgt.vito.be/artifactory/auxdata-public/worldcereal/demo/worldcereal-preprocessedinputs-Czech-demo.nc"
    import urllib.request
    urllib.request.urlretrieve(remote_url, local_file_path)

# Open the preprocessed inputs file
ds = xr.open_dataset(local_file_path)

# Get the EPSG code and convert to xarray DataArray
crs_attrs = ds["crs"].attrs
epsg = CRS.from_wkt(ds.crs.attrs["spatial_ref"]).to_epsg()  
arr = ds.drop_vars("crs").fillna(65535).astype("uint16").to_array(dim="bands")

# Inspect the data
ds

The following step derives land cover embeddings and a baseline cropland mask. The preprocessed time series (arr) are passed to a lightweight Presto model finetuned on global land cover (WorldCereal reference data) to produce 128 `landcover_embeddings` capturing multi-sensor seasonal dynamics. In the same operation, the default WorldCereal global cropland classifier is applied, yielding `cropland_classification` bands. We will use the resulting cropland mask later to optionally constrain custom crop type inference.

In [ ]:
from notebook_utils.local_inference import run_cropland_mapping, classification_to_geotiff

landcover_embeddings, cropland_classification = run_cropland_mapping(arr, epsg=epsg)

cropland_path = Path("./local_inference/cropland_classification.tif")
cropland_path.parent.mkdir(exist_ok=True)
classification_to_geotiff(
    cropland_classification,
    epsg,
    cropland_path)

Let's visualize the cropland mask you just computed!

In [ ]:
from notebook_utils.visualization import visualize_product

visualize_product(cropland_path, product='cropland', interactive_mode=True)

The next step is also exciting! 🎉 We’ll use your custom crop type classifier to generate crop type embeddings and classifications. <br>
Here's what happens:

1. **Extract Crop Type Embeddings**: Using the same seasonal time series (arr) and spatial reference (epsg), we generate 128 crop-specific embeddings (`croptype_embeddings`) aligned with your selected crop season.
2. **Run Your Classifier**: Your uploaded CatBoost model (model_url) predicts per-class probabilities and the final classification (`croptype_classification`).

Key Highlights:

- Input: Same preprocessed stack as cropland -> lightweight!
- Output: Class ID, confidence, and probabilities for each crop type.
- Pro Tip: Check class probabilities! Low confidence in large areas? It might mean season mismatch or not enough training samples. Adjust and try again! 🚀

In [ ]:
from notebook_utils.local_inference import run_croptype_mapping

croptype_embeddings, croptype_classification = run_croptype_mapping(arr, epsg=epsg, classifier_url=model_url)

In [ ]:
from notebook_utils.local_inference import classification_to_geotiff

# Set all croptype_classification pixel values to 254 where cropland_classification 'classification' band == 0
mask = cropland_classification.sel(bands="classification") == 0
croptype_classification = croptype_classification.where(~mask, 254)

# save to GeoTIFF
croptype_path = Path("local_inference") / "croptype_classification_v2.tif"
classification_to_geotiff(
    classification=croptype_classification,
    epsg=epsg,
    out_path=croptype_path
)


Let's again visualize the output classification!

In [ ]:
from worldcereal.utils.models import load_model_lut
from notebook_utils.visualization import visualize_product

lut = load_model_lut(model_url)
visualize_product(croptype_path, product='croptype', lut=lut, interactive_mode=False)

Did you get a reasonable outcome?<br>

Here is an image of the same area with some reference data, showing:
- maize in red
- soybean in yellow
- winter wheat in purple

![](./resources/Cze_exercise_result.png)

### 5. BONUS: apply your model anywhere!

Once you are happy with the performance of your model - both based on the metrics as well as from a visual check based on local inference - you are ready to create a wall-to-wall crop type map on the cloud! The steps below guide you through the process.

**Step 1: Select your area of interest (AOI)**

Provide a bounding box specifying the region for which you would like to create your map.<br>
Of course this should be an area that is representative for the model you have trained.<br>
Either draw a box directly on the map using the rectangle button, or upload a vector file (either zipped shapefile or GeoPackage) delineating your AOI.

<div class="alert alert-block alert-info">
<b>Processing area:</b><br> 
The WorldCereal system is currently optimized to process <b>20 x 20 km</b> tiles.<br>
In case your AOI exceeds this area, it will be automatically split, creating multiple map generation jobs.

We ALWAYS recommend you to select a small area to start with, whenever trying out a model for the first time!

A run of 400 km² will typically consume 40 credits and last around 20 mins.<br>
</div>

In [ ]:
from worldcereal.utils.map import ui_map

# We limit to a small area for faster processing
map = ui_map(area_limit=1200) # area_limit in km²

**Step 2: Select your year and season of interest**

We always recommend to select a similar growing season as compared to the season for which you trained your model.<br>

In case your training data was restricted to 1 or 2 years, applying your model to other years will likely result in lower quality maps.

In [ ]:
from notebook_utils.dateslider import date_slider

processing_slider = date_slider()

**Step 3: Set processing parameters**

In [ ]:
# Do you want to automatically mask temporary crops using the default WorldCereal temporary crops mask?
# By default yes.
mask_cropland = True

# In case you set mask_cropland to True, choose whether you want to store the cropland mask as separate output
save_mask = True

# Choose whether or not you want to spatially clean the classification results
postprocess_result = True

# Choose the postprocessing method you want to use ["smooth_probabilities", "majority_vote"]
# ("smooth_probabilities will do limited spatial cleaning,
# while "majority_vote" will do more aggressive spatial cleaning, depending on the value of kernel_size)
postprocess_method = "majority_vote"

# Additional parameter for the majority vote method
# (the higher the value, the more aggressive the spatial cleaning,
# should be an odd number, not larger than 25, default = 5)
kernel_size = 5

# Do you want to save the intermediate results? (before applying the postprocessing)
save_intermediate = True

# Do you want to save all class probabilities in the final product?
# If set to False, you will only get the final classification label and confidence of the winning class per pixel
keep_class_probs = True

**Step 4: Start map production**

The next cell takes care of splitting your area of interest into small tiles (size is specified through `tile_resolution` parameter) and generate a map for each tile.<br>

You will be able to track progress through the automated reporting.<br>

Results will be automatically saved to a folder containing your model name:<br> `runs/CROPTYPE_custom_{your_modelname}_{timestamp}`<br>

The first time you run this, you will be asked to authenticate with your CDSE account by clicking the link provided below the cell.<br>

<div class="alert alert-block alert-warning">
<b>What to do in case of interruption?</b><br> 
In case processing got interrupted, just make sure to manually set `output_dir` to the directory you previously used. In this case, processing will just continue where it stopped.
</div>

In [ ]:
import pandas as pd
from worldcereal.job import PostprocessParameters
from worldcereal.job import WorldCerealProductType, CropTypeParameters
from notebook_utils.production import start_production_process, monitor_production_process

# The output directory is named after the model
timestamp = pd.Timestamp.now().strftime("%Y%m%d-%H%M%S")
output_dir = Path('./runs') / f'CROPTYPE_custom_{modelname}_{timestamp}'
print(f"Output directory: {output_dir}")

# Get all postprocessing parameters
postprocess_parameters = PostprocessParameters(
    enable=postprocess_result,
    method=postprocess_method,
    kernel_size=kernel_size,
    save_intermediate=save_intermediate,
    keep_class_probs=keep_class_probs,
)

# Initializes default parameters
parameters = CropTypeParameters()

# Update parameters with user-defined values
parameters.classifier_parameters.classifier_url = model_url
parameters.save_mask = save_mask
parameters.mask_cropland = mask_cropland

# Get processing period and area
processing_period = processing_slider.get_selected_dates()
processing_extent = map.get_extent(projection='latlon')
tile_resolution = 20   # in km

args = (processing_extent, processing_period, output_dir)
kwargs = dict(
    tile_resolution=tile_resolution,
    product_type=WorldCerealProductType.CROPTYPE,
    croptype_parameters=parameters,
    postprocess_parameters=postprocess_parameters,
)

proc, queue, stop_event = start_production_process(args, kwargs)
status_df = monitor_production_process(proc, queue, stop_event)

**Step 5: Create merged product**

Once production across your tiles is finalized, you can use the cell below to merge the different tiles together into one map.<br>

In [ ]:
from notebook_utils.production import merge_maps

merged_path = merge_maps(output_dir, product='croptype')
print(f"Results merged to {merged_path}")

**Step 6: Inspect your map**

Up to four products are generated:
- `croptype-raw` --> your custom crop type product
- `croptype` --> your custom crop type product after post-processing
- `cropland-raw` --> cropland mask produced using the global WorldCereal cropland model
- `cropland` --> cropland mask, after post-processing

For each of these products, you will get a raster file containing at least two bands:
1. The label of the winning class
2. The probability of the winning class [50 - 100]
3. and beyond (optional, depending on settings): Class probabilities of each class

You can use the next cell to quickly visualize your crop type product in this notebook.

<div class="alert alert-block alert-info">
<b>Supported visualization modes:</b><br>
By default, your product is shown using matplotlib for quick visual inspection.<br>
By setting "interactive_mode" to True, both your classification and probability layers will be visualized in an interactive ipyleaflet window. By clicking the upper-right icon, followed by the button with 3 horizontal lines, you can toggle on/off individual layers and play around with layer transparency.

NOTE in order for the interactive mode to work in a VSCode environment, you need to switch on port forwarding for port 8889.
</div>

In [ ]:
from notebook_utils.visualization import visualize_product
from worldcereal.utils.models import load_model_lut

lut = load_model_lut(model_url)
visualize_product(merged_path, product='croptype', lut=lut, interactive_mode=False)

Congratulations, you have reached the end of this exercise!

Want to know more after this session?

- Check the [WorldCereal documentation](https://worldcereal.github.io/worldcereal-documentation/)
- Ask questions on our [user forum](https://forum.esa-worldcereal.org/)
- Subscribe to our [newsletter](https://esa-worldcereal.org/en#news) to get the latest updates
- Follow us on [LinkedIn](https://www.linkedin.com/company/esa-worldcereal)!